This notebook reimplements the CNN network that mentions on the paper [AI-ECGs]('https://www.thelancet.com/action/showPdf?pii=S0140-6736%2819%2931721-0'). The implementation aims to understand more detail the CNN network. Because there is no available dataset, the network is still not verified. 

Any ideas for improving as well as verifying the architecture of the network is greatly welcome.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.io import loadmat

from keras import applications
from keras import optimizers
from keras.models import Input, Sequential, Model, load_model
from keras.activations import relu
from keras.layers import Dense, Dropout, Flatten, Conv1D, Conv2D, MaxPool1D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import ReLU, ZeroPadding1D, Add, Concatenate, Reshape
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint
import keras.backend as K

C:\Users\congt\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Data import

### Build network

In [11]:
def build_temporal(ecg_input):

    # ResBlock 1
    # Residual
    x0 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu', padding='same')(ecg_input)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(ecg_input)
    x = ReLU()(x)
    x = Conv1D(filters=16, kernel_size=7, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=16, kernel_size=7, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # ResBlock 2
    # Residual
    x0 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=16, kernel_size=7, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=16, kernel_size=7, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # ResBlock 3
    x0 = Conv1D(filters=16, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=16, kernel_size=7, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=16, kernel_size=7, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])

    # Dropout 1
    x = Dropout(0.2)(x)

    # ResBlock 4
    x0 = Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=32, kernel_size=5, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=32, kernel_size=5, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # ResBlock 5
    x0 = Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=32, kernel_size=5, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=32, kernel_size=5, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # ResBlock 6
    x0 = Conv1D(filters=32, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=32, kernel_size=5, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=32, kernel_size=5, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # Dropout 2
    x = Dropout(0.2)(x)

    # ResBlock 7
    x0 = Conv1D(filters=64, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=64, kernel_size=3, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # ResBlock 8
    x0 = Conv1D(filters=64, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=64, kernel_size=3, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # ResBlock 9
    x0 = Conv1D(filters=64, kernel_size=1, strides=1, activation='relu', padding='same')(x)
    x0 = MaxPool1D(pool_size=(2))(x0)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters=64, kernel_size=3, strides=2, activation='relu', padding='same')(x)
    x = Add()([x, x0])
    # Dropout 3
    x = Dropout(0.2)(x)
    
    model = Model(ecg_input, x)
    
    return model

In [19]:
length_data = 5120
ecg_input_00 = Input(shape=(length_data, 1))
ecg_input_01 = Input(shape=(length_data, 1))
ecg_input_02 = Input(shape=(length_data, 1))
ecg_input_03 = Input(shape=(length_data, 1))
ecg_input_04 = Input(shape=(length_data, 1))
ecg_input_05 = Input(shape=(length_data, 1))
ecg_input_06 = Input(shape=(length_data, 1))
ecg_input_07 = Input(shape=(length_data, 1))

submodel_00 = build_temporal(ecg_input_00)
submodel_01 = build_temporal(ecg_input_01)
submodel_02 = build_temporal(ecg_input_02)
submodel_03 = build_temporal(ecg_input_03)
submodel_04 = build_temporal(ecg_input_04)
submodel_05 = build_temporal(ecg_input_05)
submodel_06 = build_temporal(ecg_input_06)
submodel_07 = build_temporal(ecg_input_07)

submodelout_0 = Reshape((10, 1, 64))(submodel_00.output)
submodelout_1 = Reshape((10, 1, 64))(submodel_01.output)
submodelout_2 = Reshape((10, 1, 64))(submodel_02.output)
submodelout_3 = Reshape((10, 1, 64))(submodel_03.output)
submodelout_4 = Reshape((10, 1, 64))(submodel_04.output)
submodelout_5 = Reshape((10, 1, 64))(submodel_05.output)
submodelout_6 = Reshape((10, 1, 64))(submodel_06.output)
submodelout_7 = Reshape((10, 1, 64))(submodel_07.output)

# Convolution by lead axis
merge_layer = Concatenate(axis=2)([submodelout_0, submodelout_1, submodelout_2, submodelout_3, 
                                   submodelout_4, submodelout_5, submodelout_6, submodelout_7])

x = Conv2D(filters=128, kernel_size=(1,8), strides=1, activation='relu', padding='valid')(merge_layer)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(2, actiavtion='sigmoid')(x)

input_all = [ecg_input_00, ecg_input_01, ecg_input_02, ecg_input_03, ecg_input_04 ,ecg_input_05, ecg_input_06, ecg_input_07]

final_model = Model(input_all, x)

final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 5120, 1)      0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 5120, 1)      0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 5120, 1)      0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 5120, 1)      0                                            
__________________________________________________________________________________________________
input_23 (

### Optimizer

In [55]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
final_model.compile(loss='binary_crossentropy', optimizer=sgd)